In [1]:
import get_doi_from_pubmed as gd
import pandas as pd

In [2]:
covid = ['SARS-CoV-2', 'COVID-19', 'coronavirus', 'SARS-CoV', 'MERS-CoV',
        'SARS']
antibody = ['antibody', 'antibodies', 'nanobody', 'MAb', 'immunoglobulin',
            'nanobodies']
interaction = ['neutralizing', 'neutralize', 'neutralization', 'bind',
               'binding', 'inhibit', 'targeting']
extra = ['heavy chain',  'complementarity determining region',
        'gene', 'epitope', 'receptor-binding domain', 'rbd',
        'spike protein', 'VHH']

In [11]:
papers_and_preprints = gd.pubmed_papers_and_pt(txt=False, jsonl=False, csv=True)